In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import pickle

In [ ]:
test_neural = np.load('gdrive/MyDrive/CS626/Project/Data/test_neural_features.npy')
test_statistical = np.load('gdrive/MyDrive/CS626/Project/Data/test_statistical_features.npy')
test_external = np.load('gdrive/MyDrive/CS626/Project/Data/test_external_features.npy')

In [ ]:
stance_to_index = {'agree':0,'disagree':1,'discuss':2,'unrelated':3}
index_to_stance = {0:'agree',1:'disagree',2:'discuss',3:'unrelated'}

In [ ]:
df_test = pd.read_csv('gdrive/MyDrive/CS626/Project/Data/test_Set.csv')
Y_test = df_test[['Stance']].values.tolist()
Y_res = Y_test
print(len(Y_test))

In [ ]:
def to_categorical(Y,num_classes):
  final_Y = np.zeros((len(Y),num_classes))
  i=0
  for stance in Y:
    final_Y[i][stance_to_index[stance[0]]] = 1
    i+=1
  return final_Y

In [ ]:
Y_test = to_categorical(Y_test, num_classes=4)

In [ ]:
model = pickle.load(open('gdrive/MyDrive/CS626/Project/Data/final_model_sig.pkl', 'rb'))

In [ ]:
scores = model.evaluate([test_neural,test_external,test_statistical], Y_test) 
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

In [ ]:
def ret_tags(sequences, index):
    tags = []
    for sequence in sequences:
        temp = index[np.argmax(sequence)]
        
        #temp.append(index[np.argmax(sequence)])
 
        tags.append(temp)
 
    return tags

In [ ]:
y_pred = model.predict(x = [test_neural,test_external,test_statistical])
predictions = ret_tags(y_pred, {i: t for t, i in stance_to_index.items()})

In [ ]:
Y_res = [item for sublist in Y_res for item in sublist]

In [ ]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm

In [ ]:
score, cm = score_submission(Y_res,predictions)

In [ ]:
print(score)

In [ ]:
print(np.array(cm))